<small>Copyright 2025 Amazon.com, Inc. or its affiliates. All Rights Reserved.<br>
This is AWS Content subject to the terms of the Customer Agreement</small>

# Module 1.2: Video Generation Pipeline Walkthrough

This notebook demonstrates the key steps in ROBO-Reviewer video generation pipeline. This pipeline automates the entire process from concept to video, leveraging Nova Reel 1.1 to transform your rough ideas into polished video content.

## A. Overview

The video generation pipeline consists of four main steps:
1. **Generate Prompts** - Create optimized prompts using Claude
2. **Start Video Generation** - Initiate async video generation jobs
3. **Monitor Status** - Check job progress and completion
4. **Complete Pipeline** - End-to-end video generation


## B. Install Dependencies

First, let's install the required packages for this notebook.

In [ ]:
!pip install -q matplotlib opencv-python Pillow tqdm

## C. Setup and Imports

In [9]:
import json
import time
from botocore.exceptions import ClientError
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep
import random
from retrying import retry 

## D. Configuration

Before we start generating videos, let's configure our parameters. Each setting controls a specific aspect of the video generation process:

- S3 bucket will be automatically detected from CloudFormation or fallback to config.json
- <font color='red'>Feel free to change the `USER_REQUEST` in the cell below</font>

### Create S3 bucket 

In [24]:
suffix = random.randrange(200, 900)
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_runtime = boto3_session.client('bedrock-runtime', region_name=region_name)
service = 'aoss'
s3_client = boto3.client('s3')
account_id = sts_client.get_caller_identity()["Account"]
s3_suffix = f"{region_name}-{account_id}"
S3_BUCKET = f'bedrock-kb-{s3_suffix}' # replace it with your bucket name.
pp = pprint.PrettyPrinter(indent=2)

In [25]:
# Check if bucket exists, and if not create S3 bucket for knowledge base data source
try:
    s3_client.head_bucket(Bucket=bucket_name)
    print(f'Bucket {bucket_name} Exists')
except ClientError as e:
    print(f'Creating bucket {bucket_name}')
    if region_name == "us-east-1":
        S3_BUCKET = s3_client.create_bucket(
            Bucket=bucket_name)
    else:
        S3_BUCKET = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={ 'LocationConstraint': region_name }
    )

Bucket bedrock-kb-us-east-1-442579209021 Exists


In [26]:
S3_BUCKET

'bedrock-kb-us-east-1-442579209021'

In [27]:
# AWS Configuration
#session = boto3.Session()
#bedrock_runtime = session.client('bedrock-runtime')

# Import S3 bucket configuration utility
#from utils.config import get_s3_bucket

# Get S3 bucket name
#S3_BUCKET = '' #get_s3_bucket(session)

# Video generation parameters

# Your creative concept
USER_REQUEST = "A cat playing with a ball of yarn in a cozy living room"

# Video length: must be 6 or multiples of 6 (6, 12, 18... up to 120)
DURATION_SECONDS = 6  
DURATION_SECONDS_LONG = 30

In [28]:
#S3_BUCKET = "demo-real-nova"

## E. Step 1 - Generate Optimized Prompts

The first step transforms your rough idea into professional video prompts. The `generate_prompts` function acts as an intelligent prompt engineer, using Nova Premier to:

**Key Operations:**
1. **Analyze video duration** - Determines if we need short (≤6s) or long video prompts
2. **Apply camera motion vocabulary** - Selects from 17 professional camera movements
3. **Structure prompts** - Creates descriptive captions with technical specifications
4. **Retry logic** - Handles API failures with exponential backoff
5. **Parse results** - Extracts valid JSON arrays from LLM's response

**Why this matters:** Raw user requests like 'cat playing' become cinematic prompts with proper lighting, camera work, and technical specs that Nova Reel can execute effectively.

In [29]:
def generate_prompts(
    bedrock_runtime,
    user_request,
    num_prompts = 1,
    video_duration = 6,
    model_id="us.amazon.nova-premier-v1:0"
    ):
    """
    Generate n video prompts for Amazon Nova Reel based on a given scenario.
    
    Args:
        bedrock_runtime: AWS Bedrock runtime client
        user_request (str): The video generation request from the user about what types of video to be generated.
        num_prompts: the number of prompts to be generated.
        video_duration: the video duration in seconds
        model_id (str): model ID for prompt generation
        
    Returns:
        list: List of generated video prompts
    """
    long_video_flag = False
    num_scenes = 1
    if video_duration > 6:
        long_video_flag = True
        num_scenes = video_duration // 6
    print(f"num_scenes: {num_scenes}")
    print(f"long video?: {long_video_flag}")
    
    # Camera motion dictionary
    camera_motions = {
        'aerial shot': ['Aerial shot', 'Drone shot', 'FPV drone shot', 'First person view drone shot'],
        'arc shot': ['Arc shot', '360 degree shot', '360 tracking shot', 'Orbit shot'],
        'clockwise rotation': ['Clockwise rotating shot', 'Camera rotates clockwise', 'Camera rolls clockwise'],
        'counterclockwise rotation': ['Counterclockwise rotating shot', 'Camera rotates counterclockwise', 'Camera rolls counterclockwise'],
        'dolly in': ['Dolly in shot', 'Dolly in', 'Camera moves forward', 'Camera moving forward'],
        'dolly out': ['Dolly out shot', 'Dolly out', 'Camera moves backward', 'Camera moving backward'],
        'pan left': ['Pan left shot', 'Pan left', 'Camera pans left', 'Camera moves left'],
        'pan right': ['Pan right shot', 'Pan right', 'Camera pans right', 'Camera moves right'],
        'whip pan': ['Whip pan left', 'Whip pan right'],
        'pedestal down': ['Pedestal down shot', 'Pedestal down', 'Camera moves down', 'Camera moving down'],
        'pedestal up': ['Pedestal up shot', 'Pedestal up', 'Camera moves up', 'Camera moving up'],
        'static shot': ['Static shot', 'Fixed shot'],
        'tilt down': ['Tilt down shot', 'Tilt down', 'Camera tilts down', 'Camera moving down'],
        'tilt up': ['Tilt up shot', 'Tilt up', 'Camera tilts up', 'Camera moving up'],
        'whip tilt': ['Whip tilt up', 'Whip tilt down'],
        'track left': ['Track left', 'Truck left', 'Camera tracks left', 'Camera moving to the left'],
        'track right': ['Track right', 'Truck right', 'Camera tracks right', 'Camera moving to the right'],
        'zoom in': ['Zoom in', 'Camera zooms in', 'Camera moves forward'],
        'zoom out': ['Zoom out', 'Camera zooms out', 'Camera moves backward'],
        'whip zoom': ['Whip zoom in', 'Whip zoom out'],
        'dolly zoom': ['Dolly zoom', 'Dolly zoom shot', 'Dolly zoom effect'],
        'following shot': ['Following shot']
    }
    
    # Refined prompt template for Claude
    prompt_template_short = f"""Generate {num_prompts} different prompts for the Amazon Nova Reel video generation model based on the given request. Nova Reel generates video clips from text descriptions.

<scenario>
{user_request}
</scenario>

<guidelines>
Video generation prompts should be descriptive captions, not commands. Include details about subject, action, environment, lighting, style, and camera motion.

Requirements:
- Prompts must be no longer than 512 characters.
- Place camera movement descriptions at start or end of prompt
- Avoid negation words (no, not, without)
- Write as scene descriptions, not instructions

Camera motions available: {json.dumps(camera_motions, indent=2)}
</guidelines>

<examples>
Cinematic dolly shot of a juicy cheeseburger with melting cheese, fries, and a condensation-covered cola on a worn diner table. Natural lighting, visible steam and droplets. 4k, photorealistic, shallow depth of field.

Arc shot on a salad with dressing, olives and other vegetables; 4k; Cinematic.

First person view of a motorcycle riding through the forest road.

Closeup of a large seashell in the sand. Gentle waves flow around the shell. Camera zoom in.

Clothes hanging on a thread to dry, windy; sunny day; 4k; Cinematic; highest quality;

Slow cam of a man middle age; 4k; Cinematic; in a sunny day; peaceful; highest quality; dolly in;

A mushroom drinking a cup of coffee while sitting on a couch, photorealistic.
</examples>

Generate {num_prompts} unique prompts for the scenario. Each prompt should:
1. Be clear and concise (under 512 characters)
2. Include appropriate camera motion at beginning or end
3. Describe the scene naturally (avoid starting with verbs)
4. Provide sufficient context for video generation

Return only a Python list of the {num_prompts} prompts, no additional text.
"""

    prompt_template_long = f"""Generate {num_prompts} different prompts for the Amazon Nova Reel video generation model based on the given request. Nova Reel generates video clips from text descriptions.

<scenario>
{user_request}
</scenario>

<guidelines>
Video generation prompts should be descriptive captions, not commands. Include details about subject, action, environment, lighting, style, and camera motion.

Requirements:
- Each prompt must be no longer than 4000 characters.
- Each prompt must contain {num_scenes + 1} sentences.
- Each sentence except the last one in the prompt represented a scene in the generated video. The last sentence contains the technical specifications, such as 4k, photorealistic, Cinematic.
- Place camera movement descriptions at start or end of the sentence. But this is optional.
- Avoid negation words (no, not, without)
- Write as scene descriptions, not instructions

Camera motions available: {json.dumps(camera_motions, indent=2)}
</guidelines>

<examples>
Norwegian fjord with still water reflecting mountains in perfect symmetry. Uninhabited wilderness of Giant sequoia forest with sunlight filtering between massive trunks. Sahara desert sand dunes with perfect ripple patterns. Alpine lake with crystal clear water and mountain reflection. Ancient redwood tree with detailed bark texture. Arctic ice cave with blue ice walls and ceiling. Bioluminescent plankton on beach shore at night. Bolivian salt flats with perfect sky reflection. Bamboo forest with tall stalks in filtered light. Cherry blossom grove against blue sky. Lavender field with purple rows to horizon. Autumn forest with red and gold leaves. Tropical coral reef with fish and colorful coral. Antelope Canyon with light beams through narrow passages. Banff lake with turquoise water and mountain backdrop. Joshua Tree desert at sunset with silhouetted trees. Iceland moss- covered lava field. Amazon lily pads with perfect symmetry. Hawaiian volcanic landscape with lava rock. New Zealand glowworm cave with blue ceiling lights. 8K nature photography, professional landscape lighting, no movement transitions, perfect exposure for each environment, natural color grading.

Explosion of colored powder against black background. Start with slow-motion closeup of single purple powder burst. Dolly out revealing multiple powder clouds in vibrant hues colliding mid-air. Track across spectrum of colors mixing: magenta, yellow, cyan, orange. Zoom in on particles illuminated by sunbeams. Arc shot capturing complete color field. 4K, festival celebration, high-contrast lighting
</examples>

Generate {num_prompts} unique prompts for the scenario. Each prompt should:
1. Be clear and concise (under 4000 characters)
2. Include appropriate camera motion at beginning or end
3. Describe the scene naturally (avoid starting with verbs)
4. Provide sufficient context for video generation

Return only a Python list of the {num_prompts} prompts, no additional text.
"""

    if long_video_flag:
        input_prompt = prompt_template_long
    else:
        input_prompt = prompt_template_short


    retry_delays = [1, 2, 4, 8, 16]
    messages = [
    {"role": "user",
        "content": [{"text": input_prompt}]}
    ]
    for attempt, delay in enumerate(retry_delays + [None]):
        print(f"inovke LLM - attempt {attempt}")
        try:
            response = bedrock_runtime.converse(
                modelId=model_id,
                messages=messages,
                inferenceConfig={"temperature": 0},
            )
            generated_text = response['output']['message']['content'][0]["text"]
            break
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
            if delay is not None:
                time.sleep(delay)
            else:
                return ""



    
    # Extract the Python list from the response
    try:
        # Find the list in the response text
        start_idx = generated_text.find('[')
        end_idx = generated_text.rfind(']') + 1
        list_str = generated_text[start_idx:end_idx]
        return_prompts = json.loads(list_str)  # Validate JSON
        return return_prompts[0:num_prompts]
    # raise error if failed to generate prompts
    except Exception as e:
        raise ValueError(f"Error parsing generated text: {e}" + "Failed to generate prompts. Please try again.")

In [30]:
# Generate prompts using Claude
print("Generating video prompts for short video...")
prompts_short = generate_prompts(
    bedrock_runtime=bedrock_runtime,
    user_request=USER_REQUEST,
    num_prompts=1,
    video_duration=DURATION_SECONDS,
    model_id="us.amazon.nova-premier-v1:0"
)

print(f"\nGenerated {len(prompts_short)} prompts:")
for i, prompt in enumerate(prompts_short, 1):
    print(f"{i}. {prompt}")


print("\nGenerating video prompts for long video...")
prompts_long = generate_prompts(
    bedrock_runtime=bedrock_runtime,
    user_request=USER_REQUEST,
    num_prompts=1,
    video_duration=DURATION_SECONDS_LONG,
    model_id="us.amazon.nova-premier-v1:0"
)

print(f"\nGenerated {len(prompts_long)} prompts:")
for i, prompt in enumerate(prompts_long, 1):
    print(f"{i}. {prompt}")



Generating video prompts for short video...
num_scenes: 1
long video?: False
inovke LLM - attempt 0

Generated 1 prompts:
1. Softly lit living room with plush furniture. A curious cat bats at a colorful yarn ball, unraveling it playfully. Sunlight streams through sheer curtains. Camera pans right, capturing the feline's joyful antics.

Generating video prompts for long video...
num_scenes: 5
long video?: True
inovke LLM - attempt 0

Generated 1 prompts:
1. Soft morning light streams through sheer curtains in a cozy living room. A fluffy grey cat sits on a plush rug, batting playfully at a tangled ball of yarn. Camera pans right to show the cat chasing the rolling yarn across wooden floors. The feline pounces with precision, wrapping itself in the yarn. Camera zooms in to capture whiskers twitching in delight. Rendered in 4K with warm, photorealistic lighting and smooth cinematic motion.


## F. Step 2 - Start Video Generation Jobs

Now we take our optimized prompts and submit them to Nova Reel for video generation. The `start_invocation_t2v` function handles the complexity of AWS Bedrock's async API:

**Key Operations:**
1. **Task type selection** - Uses `TEXT_VIDEO` for short videos (≤6s) or `MULTI_SHOT_AUTOMATED` for longer videos
2. **Configuration setup** - Applies video parameters (resolution, FPS, duration)
3. **Seed management** - Increments seeds for each prompt to ensure variety
4. **Async invocation** - Starts non-blocking video generation jobs
5. **ARN collection** - Returns job identifiers for status tracking

**Why async matters:** Video generation takes 3-5 minutes per video. Async processing lets us start multiple jobs simultaneously and do other work while they render.

In [31]:
def start_invocation_t2v(
    bedrock_runtime,
    s3_bucket:str,
    text_prompts:list,
    duration_seconds:int = 6,
    fps:int = 24,
    dimension:str = "1280x720",
    seed:int = 42,
    model_id:str = "amazon.nova-reel-v1:1"
    ):
    """
    Start invocations to generate videos from text prompts.

    Parameters:
    -----------
    bedrock_runtime: boto3.client
        The Bedrock Runtime client.

    s3_bucket: str
        The S3 bucket where the video will be stored.

    text_prompt: list
        The text prompt(s) to generate video(s).

    duration_seconds: int
        The duration of the video in seconds. Default is 6.

    fps: int
        The frames per second of the video. Default is 24.

    dimension: str
        The dimension of the video. Default is "1280x720".

    seed: int
        The seed to use for the video generation. Default is 0.

    model_id: str
        The model ID to use for the video generation. Default is 
        "amazon.nova-reel-v1:1".

    Returns:
    --------
    str or list
        The invocation ARN(s).
    
    
    """
    
    # Handle both single prompt and list of prompts
    prompts = text_prompts
    
    invocation_arns = []

    if duration_seconds <= 6:
        
        for i, prompt in enumerate(prompts):
            skip_flag = False
            model_input = {
                "taskType": "TEXT_VIDEO",
                "textToVideoParams": {
                    "text": prompt
                },
                "videoGenerationConfig": {
                    "durationSeconds": duration_seconds,
                    "fps": fps,
                    "dimension": dimension,
                    "seed": seed + i,  # Increment seed for each prompt
                },
            }

            retry_delays = [32, 32, 32, 32, 32]
            for attempt, delay in enumerate(retry_delays + [None]):
                print(f"inovke Nova Reel - attempt {attempt}")
                try:
                    invocation = bedrock_runtime.start_async_invoke(
                        modelId = model_id,
                        modelInput=model_input,
                        outputDataConfig={
                            "s3OutputDataConfig": {
                                "s3Uri": "s3://" + s3_bucket
                            }
                        }
                    )
                    break
                except (ClientError, Exception) as e:
                    print(f"ERROR: Reason: {e}")
                    if delay is not None:
                        print(f"Retryinng in {delay} seconds.")
                        time.sleep(delay)
                    else:
                        skip_flag = True

            if not skip_flag:
                invocation_arns.append(invocation["invocationArn"])
    else:

        for i, prompt in enumerate(prompts):
            skip_flag = False
            model_input = {
                "taskType": "MULTI_SHOT_AUTOMATED",
                "multiShotAutomatedParams": {
                    "text": prompt
                },
                "videoGenerationConfig": {
                    "durationSeconds": duration_seconds,
                    "fps": fps,
                    "dimension": dimension,
                    "seed": seed + i,  # Increment seed for each prompt
                },
            }

            retry_delays = [32, 32, 32, 32, 32]
            for attempt, delay in enumerate(retry_delays + [None]):
                print(f"inovke Nova Reel - attempt {attempt}")
                try:
                    invocation = bedrock_runtime.start_async_invoke(
                        modelId = model_id,
                        modelInput=model_input,
                        outputDataConfig={
                            "s3OutputDataConfig": {
                                "s3Uri": "s3://" + s3_bucket
                            }
                        }
                    )
                    break
                except (ClientError, Exception) as e:
                    print(f"ERROR: Reason: {e}")
                    if delay is not None:
                        print(f"Retryinng in {delay} seconds.")
                        time.sleep(delay)
                    else:
                        skip_flag = True

            if not skip_flag:
                invocation_arns.append(invocation["invocationArn"])
    
    return invocation_arns

In [32]:
# Start video generation jobs
print("Starting video generation jobs...")
invocation_arns = start_invocation_t2v(
    bedrock_runtime=bedrock_runtime,
    s3_bucket=S3_BUCKET,
    text_prompts=prompts_short,
    duration_seconds=DURATION_SECONDS
)

print(f"\nStarted {len(invocation_arns)} video generation jobs:")
for i, arn in enumerate(invocation_arns, 1):
    print(f"{i}. {arn}")

invocation_arns_2 = start_invocation_t2v(
    bedrock_runtime=bedrock_runtime,
    s3_bucket=S3_BUCKET,
    text_prompts=prompts_long,
    duration_seconds=DURATION_SECONDS_LONG
)

print(f"\nStarted {len(invocation_arns_2)} video generation jobs:")
for i, arn in enumerate(invocation_arns_2, 1):
    print(f"{i}. {arn}")

print("\n🚀 Jobs are now running asynchronously on Nova Reel infrastructure.")
print("   Each ARN is a unique identifier we'll use to track progress.")
print("   Videos will be saved to your S3 bucket when complete.")

invocation_arns = invocation_arns + invocation_arns_2

Starting video generation jobs...
inovke Nova Reel - attempt 0

Started 1 video generation jobs:
1. arn:aws:bedrock:us-east-1:442579209021:async-invoke/7zwg4d79lkwc
inovke Nova Reel - attempt 0

Started 1 video generation jobs:
1. arn:aws:bedrock:us-east-1:442579209021:async-invoke/2uaqgzv2q879

🚀 Jobs are now running asynchronously on Nova Reel infrastructure.
   Each ARN is a unique identifier we'll use to track progress.
   Videos will be saved to your S3 bucket when complete.


## G. Step 3 - Monitor Job Status

This is where we wait for the magic to happen. The `invocation_status_check` function provides intelligent monitoring of our video generation jobs:

**Key Operations:**
1. **Status tracking** - Maintains state for each job (InProgress/Completed/Failed)
2. **Polling loop** - Checks job status every 2 seconds
3. **Progress reporting** - Shows count of active jobs, detailed info for long-running jobs
4. **Error handling** - Captures and reports failure messages
5. **Result collection** - Extracts S3 URIs when videos complete

**Smart monitoring:** The function balances responsiveness (2s polling) with API courtesy, providing more detailed updates for jobs that take longer than expected.

In [33]:
def invocation_status_check(
        bedrock_runtime,
        invocation_arns
    ):
    """
    Check the status of invocation(s).

    Parameters:
    -----------
    bedrock_runtime: boto3.client
        The Bedrock Runtime client.

    invocation_arn: list
        The invocation ARN(s).

    Returns:
    --------
    str or list
        The S3 URI(s) of the generated video(s). If generation failed,
        return None for that video.
    
    """
    
    # Handle both single ARN and list of ARNs
    arns = invocation_arns
    
    # Track status for each ARN
    arn_status = {arn: "InProgress" for arn in arns}
    results = {}
    update_count = 0
    arn_in_progress_flag = {arn: False for arn in arns}
    
    while any(status not in ["Completed", "Failed"] for status in arn_status.values()):
        time.sleep(0.5)
        update_count += 1
        
        for arn in arns:
            if arn_status[arn] in ["Completed", "Failed"]:
                continue
            try:    
                invocation = bedrock_runtime.get_async_invoke(invocationArn=arn)
            except:
                continue
                
            status = invocation["status"]
            arn_status[arn] = status
            
            if status == "InProgress":
                if update_count > 25000 and update_count % 5 == 0:
                    start_time = invocation["submitTime"]
                    if not arn_in_progress_flag[arn]:
                        print(f"Job {arn} is still in progress. Started at: {start_time}")
                        arn_in_progress_flag[arn] = True
                elif update_count % 5 == 0:
                    print(f"Jobs in progress: {sum(1 for s in arn_status.values() if s == 'InProgress')}   " + "." * (update_count%30//5) + " " * (5- update_count%30//5), end = "\r")
                    
            elif status == "Failed":
                failure_message = invocation["failureMessage"]
                print(f"Job {arn} failed. Failure message: {failure_message}")
                results[arn] = None
            
            elif status == "Completed":
                bucket_uri = invocation["outputDataConfig"]["s3OutputDataConfig"]["s3Uri"]
                video_uri = bucket_uri + "/output.mp4"
                results[arn] = video_uri
    
    # Return results in same order as input
    result_list = [results.get(arn) for arn in arns]
    return result_list

In [ ]:
# Monitor job status and get results
print("Monitoring video generation progress...")

video_uris = invocation_status_check(
    bedrock_runtime=bedrock_runtime,
    invocation_arns=invocation_arns
)

print(f"\nVideo generation completed!")
print(f"Generated {len([uri for uri in video_uris if uri])} successful videos:")
for i, uri in enumerate(video_uris, 1):
    if uri:
        print(f"{i}. {uri}")
    else:
        print(f"{i}. Failed to generate")

print("✅ Success! Your videos are ready in S3.")

Monitoring video generation progress...


## Optional: Download and Display Videos

Now let's download our generated videos from S3 and display them in the notebook:

**Key Operations:**
1. **Download from S3** - Pull videos from S3 to local storage
2. **Display in notebook** - Show videos inline using HTML5 video player
3. **Handle multiple videos** - Process all successful generations

In [ ]:
import os
from IPython.display import HTML, display

def download_videos(s3_uris, s3_bucket):
    s3_client = session.client('s3')
    os.makedirs('generated_videos', exist_ok=True)
    downloaded_videos = []
    
    for i, uri in enumerate(s3_uris):
        if uri is None:
            continue
        s3_key = uri.replace(f"s3://{s3_bucket}/", "")
        local_filename = f"generated_videos/video_{i+1}.mp4"
        
        try:
            s3_client.download_file(s3_bucket, s3_key, local_filename)
            downloaded_videos.append(local_filename)
            print(f"✅ Downloaded: {local_filename}")
        except Exception as e:
            print(f"❌ Failed to download video {i+1}: {e}")
    
    return downloaded_videos

def display_video_in_notebook(video_path, width=640):
    video_html = f"""
    <video width="{width}" controls>
        <source src="{video_path}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """
    return HTML(video_html)

In [ ]:
# Download and display the generated videos
if 'video_uris' in locals() and any(video_uris):
    print("📥 Downloading videos from S3...")
    local_videos = download_videos(video_uris, S3_BUCKET)
    
    print(f"\n🎬 Displaying {len(local_videos)} generated videos:")
    
    for i, video_path in enumerate(local_videos):
        display(display_video_in_notebook(video_path))
        
        file_size = os.path.getsize(video_path) / (1024*1024)
        print(f"📁 File: {video_path} ({file_size:.1f} MB)")

else:
    print("⚠️  No videos available. Run the previous sections first.")

## H. Complete Pipeline

The `video_generation_pipeline` function demonstrates how all three steps work together seamlessly. This wrapper function:

**Key Operations:**
1. **Duration normalization** - Adjusts video length to valid Nova Reel values (6s or multiples of 6)
2. **Sequential execution** - Calls generate_prompts → start_invocation_t2v → invocation_status_check
3. **Move videos and prompts to centralized location** - All the generated videos and their corresponding prompts will be moved to s3://<your_bucket>/generated_videos/

**Output Structure**:

The final result will be saved to s3 bucket with the following structures

```
s3://<your bucket>/
└── generated_videos/
    ├── <generated video file>.mp4          # The generated video
    └── <generated video file>_prompt.txt   # The prompt used for this video
```


- <font color='red'>Feel free to change the `user_request` in the cell below</font>
- <font color='red'>Change `auto_prompt_generation` if you would like to use your input as video generation prompt directly</font>

In [31]:
user_request="I would like to generate a video that contains a Experimental Sciences Center: Physics, chemistry and biology laboratories with cutting-edge equipment, educational greenhouse and astronomical observatory."
auto_prompt_generation = True

In [32]:
from utils.video_generation import video_generation_pipeline


print("Running complete video generation pipeline...")
# generate one video with 6 seconds duration
pipeline_video_uris = video_generation_pipeline(
    boto3_session=session,
    s3_bucket=S3_BUCKET,
    user_request=user_request,
    prompt_optimization_flag = auto_prompt_generation,
    num_videos=1,
    duration_seconds=12,
)

print(f"\nPipeline completed! Generated videos:")
for i, uri in enumerate(pipeline_video_uris, 1):
    if uri:
        print(f"{i}. {uri}")
    else:
        print(f"{i}. Failed to generate")

# generate one video with 30 seconds duration
pipeline_video_uris = video_generation_pipeline(
    boto3_session=session,
    s3_bucket=S3_BUCKET,
    user_request=user_request,
    prompt_optimization_flag = auto_prompt_generation,
    num_videos=1,
    duration_seconds=30,
)

print(f"\nPipeline completed! Generated videos:")
for i, uri in enumerate(pipeline_video_uris, 1):
    if uri:
        print(f"{i}. {uri}")
    else:
        print(f"{i}. Failed to generate")

Running complete video generation pipeline...
num_scenes: 2
inovke LLM - attempt 0

Generated 1 prompts:
1. Aerial shot showcasing the Experimental Sciences Center with sleek modern architecture. Camera pans down to reveal physics labs filled with lasers and oscilloscopes, transitioning to chemistry labs with rows of colorful beakers. Finally, the camera tilts up to an astronomical observatory dome under a star-studded night sky. Render in 4k, ultra-realistic textures, cinematic lighting.


inovke Nova Reel - attempt 0
Moved video: s3://demo-real-nova/78u7yz6p0kym/output.mp4 -> s3://demo-real-nova/generated_videos/78u7yz6p0kym.mp4
Saved prompt: s3://demo-real-nova/generated_videos/78u7yz6p0kym_prompt.txt

Pipeline completed! Generated videos:
1. s3://demo-real-nova/generated_videos/78u7yz6p0kym.mp4
num_scenes: 5
inovke LLM - attempt 0

Generated 1 prompts:
1. Aerial shot of the Experimental Sciences Center at sunrise, showcasing interconnected buildings. Dolly in to the physics lab whe

## Conclusion

We have now seen how to generate videos using Nove Reel. Now you can move to the next module to see how to process the videos before evaluation.